In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03.tar.gz
!tar -xzvf "/content/ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03.tar.gz" -C "/content/"

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18.tar.gz
!tar -xzvf "/content/ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18.tar.gz" -C "/content/"

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz
!tar -xzvf "/content/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz" -C "/content/"

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz
!tar -xzvf "/content/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz" -C "/content/"

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -xzvf "/content/ssd_mobilenet_v2_coco_2018_03_29.tar.gz" -C "/content/"

In [ ]:
import tensorflow as tf
print(tf.__version__)

import numpy as np

In [ ]:
# working
model = tf.saved_model.load('/content/ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03/saved_model')
concrete_func = model.signatures[
tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape([1, 300, 300, 3])

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()

with open('detect.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# non working
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='/content/ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03/saved_model/saved_model.pb', 
    input_arrays=['image_tensor'],
    output_arrays=['detection_boxes', 'detection_classes', 'detection_scores', 'num_detections'],
    input_shapes={'image_tensor': [1, 300, 300, 3]}
)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()
with open('detect.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# print layers
def print_layers(graph_def):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph

    print("-" * 50)
    print("Frozen model layers: ")
    layers = [op.name for op in import_graph.get_operations()]
    ops = import_graph.get_operations()
    print(ops[0])
    print("Input layer: ", layers[0])
    print("Output layer: ", layers)
    print("-" * 50)

# Load frozen graph using TensorFlow 1.x functions
with tf.io.gfile.GFile("/content/ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03/frozen_inference_graph.pb", "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    loaded = graph_def.ParseFromString(f.read())

frozen_func = print_layers(graph_def=graph_def)

Below clone tensorflow and use bazel to run command

In [ ]:
!git clone https://github.com/tensorflow/tensorflow.git
#!git clone https://github.com/tensorflow/models.git
#!git clone --branch archive https://github.com/tensorflow/models.git

In [ ]:
# Download Latest version of Bazelisk
!wget https://github.com/bazelbuild/bazelisk/releases/latest/download/bazelisk-linux-amd64

# Make script executable
!chmod +x bazelisk-linux-amd64

# Adding to the path
!sudo mv bazelisk-linux-amd64 /usr/local/bin/bazel


In [ ]:
# Bazel info.
!bazel

In [ ]:
cd tensorflow

In [ ]:
!ls

In [ ]:
!bazel run -c opt tensorflow/lite/python:tflite_convert -- \
--enable_v1_converter \
--graph_def_file='/content/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb' \
--output_file='/content/ssd_mobilenet_v2_coco_2018_03_29/detect.tflite' \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_dev_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
print(input_details)
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array([[0.195312]], dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

In [ ]:
cd ../

In [ ]:
%env CONFIG_FILE='/content/ssd_mobilenet_v2_coco_2018_03_29/pipeline.config'
%env CHECKPOINT_PATH='/content/ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta'
%env OUTPUT_DIR=/tmp/tflite

!export -p

In [ ]:
!find . -name "setup.py"

In [ ]:
#!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
!sudo python setup.py install

In [ ]:
!pip install tf_slim

In [ ]:
#!protoc -I=./ --python_out=./ ./object_detection/protos/*.proto
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!python object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=$CONFIG_FILE \
--trained_checkpoint_prefix=$CHECKPOINT_PATH \
--output_directory=$OUTPUT_DIR \
--add_postprocessing_op=true